<br>

# Introdução


In [1]:
#!pip3 install PyPDF2


In [2]:
import os
import time
import requests
import numpy as np
import pandas as pd
from PyPDF2 import PdfFileMerger, PdfFileReader


c:\Users\michel\.conda\envs\pablocarreira-py39\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.firefox import GeckoDriverManager


In [4]:
from paths import *


In [5]:
from dotenv import load_dotenv, find_dotenv, dotenv_values

# Credenciais
config = dotenv_values(find_dotenv(usecwd=True))

# Variaveis do Driver
os.environ['GH_TOKEN'] = config['GH_TOKEN']
CPF = config['CPF']
PASSWORD = config['PASSWORD']


<br>

## Create Driver


In [6]:
def create_driver():
    # Parameters
    DRIVER_PATH = driver_path
    DRIVER_PATH = None
    HEADLESS = False
    #os.environ['WDM_SSL_VERIFY'] = '0'

    # Cria Driver
    global driver
    firefox_options = FirefoxOptions()
    firefox_options.headless = HEADLESS
    driver = webdriver.Firefox(
        service=Service(
            executable_path=GeckoDriverManager(path=DRIVER_PATH).install(),
            log_path=log_path / 'geckodriver.log',
        ),
        options=firefox_options,
    )

    # Add-ons Xpath
    xpath_path = adds_path / 'xpath.xpi'
    xpath_path = xpath_path.absolute().resolve()
    if not xpath_path.is_file():
        r = requests.get(
            'https://addons.mozilla.org/firefox/downloads/file/3588871/xpath_finder-1.0.2-fx.xpi')
        with open(xpath_path, 'wb') as f:
            f.write(r.content)
    driver.install_addon(str(xpath_path), temporary=True)


In [7]:
# Cria Driver
create_driver()


[WDM] - ====== WebDriver manager ======
[WDM] - Current firefox version is 102.0
[WDM] - Get LATEST geckodriver version for 102.0 firefox
[WDM] - GH_TOKEN will be used to perform requests
[WDM] - Driver [C:\Users\michel\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


<br>

## Faz Login


In [8]:
def get_login(cpf, password):
    # Get Url
    url = 'https://e.ambiente.sp.gov.br/atendimento/'
    driver.get(url)

    try:
        # Hide Popup Tour
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@class="hopscotch-bubble animated tour-portalTour"]')))
        element = driver.find_elements(
            By.XPATH, '//*[@class="hopscotch-bubble animated tour-portalTour"]')[1]
        driver.execute_script(
            "arguments[0].style.visibility='hidden'", element)
    except:
        pass

    try:
        # Efetuar Login
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="info-usuario"]'))).click()

        # Pega o XPath do iframe e atribui a uma variável
        iframe = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))

        # Muda o foco para o iframe
        driver.switch_to.frame(iframe)

        # Entra com credenciais
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="j_username"]'))).send_keys(cpf)
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="j_password"]'))).send_keys(password)

        # Clica em entrar
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="pbEntrar"]'))).click()
    except:
        pass


In [9]:
# Login
get_login(CPF, PASSWORD)


<br>

## Go Pages


In [10]:
def fecha_popups():
    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # ddd
    try:
        tour = driver.find_element(
            By.XPATH, '//*[@class="hopscotch-bubble animated tour-portalTour"]//*[@class="hopscotch-actions"]//..')
        tour = tour.find_element(
            By.XPATH, './/*[@class="hopscotch-bubble-close hopscotch-close"]')
        tour.click()
    except:
        pass


In [11]:
fecha_popups()


In [12]:
def open_sidebar():
    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    collapsed = driver.find_element(
        By.XPATH, '//aside[contains(@class, "abas")]').get_attribute('class')
    if collapsed == 'abas collapsed':
        driver.find_element(
            By.XPATH, '//*[@title="Exibir/Esconder menu"]').click()
    return 0


In [13]:
def go_meusprocessos():
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Meus Processos"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Meus Processos"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)


In [14]:
def go_aberturaprocessos():
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Processos Liberados"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Abertura de Processos"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)


In [15]:
def go_processosliberados():
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Processos Liberados"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Processos liberados"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)


<br>

## Meus Processos


In [16]:
def get_meusprocessos():
    # Open Sidebar
    open_sidebar()

    # Go Page
    go_aberturaprocessos()
    time.sleep(3)
    go_meusprocessos()

    # Pega Table
    table = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="meusProcessos"]')))

    # Linhas
    WebDriverWait(table, 3).until(
        EC.presence_of_element_located((By.XPATH, './/tr')))
    rows = table.find_elements(By.XPATH, './/tr')

    # Loop
    list_head = []
    list_row = []
    list_processo = []
    for row in rows[:-2]:
        for cel in row.find_elements(By.XPATH, './/th'):
            list_head.append(cel.text)
        for cel in row.find_elements(By.XPATH, './/td'):
            list_row.append(cel.text)
            if cel.text == 'create_new_folder':
                list_processo.append(cel)

    # Pega Linhas de Interesse
    n_processos = len(rows[:-2])-1
    list_row = [x for x in list_row if x != '']
    list_row = np.array(list_row).reshape(n_processos, 5)

    # Dataframe
    df = pd.DataFrame(
        list_row,
        columns=list_head
    )
    df.drop(['Det.'], inplace=True, axis=1)
    return df


In [17]:
# Meus Processos
df_meusprocessos = get_meusprocessos()
df_meusprocessos.head()


,Processo,Data de entrada,Órgão/Unidade atual,Situação
0,CETESB.063415/2022-63,01/07/2022,CETESB / SETOR DE TRIAGEM E ACOMPANHAMENTO DE ...,Arquivado
1,CETESB.062331/2022-58,28/06/2022,CETESB / SETOR DE AVALIAÇÃO E GESTÃO DE RESÍDU...,Arquivado
2,CETESB.056659/2021-31,21/05/2021,CETESB / EXPEDIENTE - AGÊNCIA AMBIENTAL DE PIR...,Arquivado
3,CETESB.055286/2021-52,18/05/2021,CETESB / EXPEDIENTE - AGÊNCIA AMBIENTAL DE PIR...,Arquivado
4,CETESB.055285/2021-91,18/05/2021,CETESB / EXPEDIENTE - AGÊNCIA AMBIENTAL DE PIR...,Arquivado


<br>

## Processos Liberados


In [18]:
def get_processosliberados():
    # Open Sidebar
    open_sidebar()

    # Go Page
    go_aberturaprocessos()
    time.sleep(3)
    go_processosliberados()

    # Lista Processos
    tag = '//*[@class="sds-collapse-list sds-card liberacao-collapse-container"]//*[@class="sds-collapse-list__item"]'
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, tag)))
    list_process = driver.find_elements(By.XPATH, tag)

    list_process_text = []
    for process in list_process:
        list_process_text.append(process.text.split('\n'))

    # Create Dataframe
    df = pd.DataFrame(
        list_process_text,
        columns=['Nome', '-', 'Data de Liberação', '-',
                 'Acesso termina em', '-', 'Situação', '-']
    )
    # print(df.head())

    # Edit Dataframe
    df[['Processo', 'Interessado', 'Empresa']
       ] = df['Nome'].str.split(' - ', n=2, expand=True)

    df = df[['Processo', 'Interessado', 'Empresa',
             'Data de Liberação', 'Acesso termina em', 'Situação']].copy()

    df['Interessado'] = df['Interessado'].replace(
        {'Interessado: ': ''}, regex=True)

    # Result
    return df


In [19]:
# Processos Liberados
df_processosliberados = get_processosliberados()
df_processosliberados.head()


,Processo,Interessado,Empresa,Data de Liberação,Acesso termina em,Situação
0,CETESB.073791/2021-28,11480200000105,CONSIMARES - CONSORCIO INTERMUNICIPAL DE MANEJ...,06/07/2022,13/07/2022,Acesso ativo
1,CETESB.009043/2020-79,15664292000134,PIRACICABA AMBIENTAL SA,21/05/2021,21/06/2021,Acesso expirado
2,CETESB.028626/2021-49,15664292000134,PIRACICABA AMBIENTAL SA,19/05/2021,19/06/2021,Acesso expirado
3,CETESB.041729/2019-05,15664292000134,PIRACICABA AMBIENTAL SA,19/05/2021,19/06/2021,Acesso expirado
4,CETESB.027048/2018-80,15664292000134,PIRACICABA AMBIENTAL SA,19/05/2021,18/06/2021,Acesso expirado


<br>

## Select Process e Small Defs


In [20]:
def get_id_liberacao_doc(url):
    return os.path.basename(url)


In [21]:
def select_processo(df_processosliberados):
    # Input
    valores = ''.join('{}'.format(list(df_processosliberados.index)))
    processo = int(input(
        'Insira o número do índice do processo. O número deve estar entre {}: '.format(valores)))

    # Check se o processo está no índice
    while not(processo in list(df_processosliberados.index)):
        print('Número não consta no índice. Por favor, insira um número válido!')
        processo = int(input(
            'Insira o número do índice do processo. O número deve estar entre {}: '.format(valores)))

    # Check situação
    processo_situacao = df_processosliberados.loc[processo, 'Situação']
    while processo_situacao != 'Acesso ativo':
        processo = int(input(
            'Insira o número do índice do processo. O número deve estar entre {}: '.format(valores)))

    # Result
    processo_text = df_processosliberados.loc[processo, 'Processo'].replace(
        '.', '_').replace('/', '-')
    print('O processo escolhido foi "{}" e está com "{}"'.format(
        processo_text, processo_situacao))
    return processo, processo_text


In [22]:
# Select Process
processo, processo_text = select_processo(df_processosliberados)


O processo escolhido foi "CETESB_073791-2021-28" e está com "Acesso ativo"


<br>

## Get List Files


In [23]:
def get_file_list(processo):
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Processos Liberados"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Processos liberados"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)
    time.sleep(2)

    # Lista Processos
    tag = '//*[@class="sds-collapse-list sds-card liberacao-collapse-container"]//*[@class="sds-collapse-list__item"]'
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, tag)))
    list_process = driver.find_elements(By.XPATH, tag)

    # Minimiza todos os processos
    for process in list_process:
        try:
            tag = './/button[@class="sds-btn sds-btn--icon button-rotate"]'
            WebDriverWait(process, 1).until(
                EC.presence_of_element_located((By.XPATH, tag))).click()
            print('a')
        except:
            pass

    # Seleciona Processo
    WebDriverWait(list_process[processo], 1).until(EC.presence_of_element_located(
        (By.XPATH, './/button[@class="sds-btn sds-btn--icon"]'))).click()

    # Lista de todos os arquivos do processo
    tag = '//*[@id="secao-documentos{}"]//*[@class="sds-data-list"]//*[@class="sds-data-list__item"]'.format(
        processo)
    WebDriverWait(list_process[processo], 3).until(
        EC.presence_of_element_located((By.XPATH, tag)))
    files = driver.find_elements(By.XPATH, tag)

    # Loop
    list_name = []
    list_url = []
    for file in files:
        name = file.text
        url = file.find_element(By.XPATH,
                                './/*[@class="sds-link"]').get_attribute('href')
        list_name.append(name)
        list_url.append(url)

    # Dataframe
    df = pd.DataFrame(
        list(zip(list_name, list_url)),
        columns=['nome', 'url1']
    )
    df['nome'] = df['nome'].replace({'file_download ': ''}, regex=True)
    df['id_liberacao_doc_1'] = df.apply(
        lambda x: get_id_liberacao_doc(x['url1']), axis=1)
    return df


In [24]:
# Get File List
df = get_file_list(processo)
df.head()


,nome,url1,id_liberacao_doc_1
0,Folha Líder (pág. 1),https://e.ambiente.sp.gov.br/cpa-core-backend/...,NjA2NzM1
1,Formulário da tarefa (pág. 2 a 17),https://e.ambiente.sp.gov.br/cpa-core-backend/...,NjA2NzM2
2,Carta de apresentação.pdf (pág. 18 a 20),https://e.ambiente.sp.gov.br/cpa-core-backend/...,NjA2NzM3
3,Ficha cadastral.pdf (pág. 21 a 22),https://e.ambiente.sp.gov.br/cpa-core-backend/...,NjA2NzM4
4,Tabela Síntese do Licenciamento (Sala de cenár...,https://e.ambiente.sp.gov.br/cpa-core-backend/...,NjA2NzM5


<br>

## Get File


<br>

### Função 1: Pega outras URLs


In [31]:
def get_url_redirects(url):
    # Get URL
    driver.get(url)
    time.sleep(10)

    # TODO: Fazer download dos arquivos zipados.
    # id btnDownload

    # URL 1
    url1 = driver.current_url

    # URL 2
    url2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="content"]'))).get_attribute('src')

    # Results
    urls = list([url1, url2])
    return urls


In [32]:
# Adjust Table: Função 1
df[['url2', 'url3']] = df.apply(lambda x: get_url_redirects(
    x['url1']), axis=1, result_type='expand')


<br>

### Função 2: Pega Parâmetros


In [29]:
def adjust_url3(url):
    """
    ddddd

    """
    
    splitado = url.split(
        '&',        
    )
    print(splitado)

    go_doc_id = splitado[0].split('?')[-1]
    go_doc_id = go_doc_id.split('=')[-1]

    id_liberacao_doc = splitado[1]
    id_liberacao_doc = id_liberacao_doc.split('=')[-1]

    fl_assinado = splitado[2]
    fl_assinado = fl_assinado.split('=')[-1]

    download_url = splitado[3]
    download_url = download_url.split('=')[-1]

    return list([go_doc_id, id_liberacao_doc, fl_assinado, download_url])


In [30]:
# Adjust Table: Função 2
df[['go_doc_id', 'id_liberacao_doc_2', 'fl_assinado', 'download_url']] = df.apply(
    lambda x: adjust_url3(x['url3']), axis=1, result_type='expand')


['https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.html?goDocId=9524467', 'idLiberacaoDoc=NjA2NzM1', 'flAssinado=N', 'downloadURL=/cpavPastaExterna/services/pasta-digital/arquivo']
['https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.html?goDocId=9514984', 'idLiberacaoDoc=NjA2NzM2', 'flAssinado=N', 'downloadURL=/cpavPastaExterna/services/pasta-digital/arquivo']
['https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.html?goDocId=9471081', 'idLiberacaoDoc=NjA2NzM3', 'flAssinado=N', 'downloadURL=/cpavPastaExterna/services/pasta-digital/arquivo']
['https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.html?goDocId=9471082', 'idLiberacaoDoc=NjA2NzM4', 'flAssinado=N', 'downloadURL=/cpavPastaExterna/services/pasta-digital/arquivo']
['https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.html?goDocId=9514987', 'idLiberacaoDoc=NjA2NzM5', 'flAssinado=N', 'downloadURL=/cpavPastaExterna/services/pasta-digital/arquivo']
['https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.ht

IndexError: list index out of range

<br>

### Função 3: Get URL Download


In [ ]:
def get_url_download(go_doc_id, id_liberacao_doc):
    # Parametros Link
    dados = {
        'goDocId': go_doc_id,
        'visualizaSemEfeito': 'null',
        'flSituacaoDoc': 'null',
        'flPDF': 'S',
        'idLiberacaoDoc': id_liberacao_doc,
    }

    # URL
    ROOT = 'https://e.ambiente.sp.gov.br/cpa-core-backend/interacao/acesso-externo-documento/documento/?'
    url = ('{}goDocId={}&visualizaSemEfeito={}&flSituacaoDoc={}&flPDF={}&idLiberacaoDoc={}'.format(
        ROOT,
        dados['goDocId'],
        dados['visualizaSemEfeito'],
        dados['flSituacaoDoc'],
        dados['flPDF'],
        dados['idLiberacaoDoc'],
    ))

    return url


In [ ]:
# Função 3
df.loc[:, 'url_download'] = df.apply(lambda x: get_url_download(
    x['go_doc_id'], x['id_liberacao_doc_1']), axis=1)


In [ ]:
# Reordena Coluna
df = df[[
    'nome',
    'url1',
    'url2',
    'url3',
    'url_download',
    'go_doc_id',
    'id_liberacao_doc_1',
    'id_liberacao_doc_2',
    'fl_assinado',
    'download_url'
]].copy()

# Create Directory
processo_path = data_path / processo_text
processo_path.mkdir(exist_ok=True)

# Escreve Tabela
df.to_csv(
    data_path / 'tab_arquivos.csv',
    index=False
)


In [ ]:
# Função 1
#url = 'https://e.ambiente.sp.gov.br/cpa-core-backend/interacao/acesso-externo-documento/MjczMzM5'
#urls = get_url_redirects(url)

# Função 2
#url = 'https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.html?goDocId=8083252&idLiberacaoDoc=MjczMzQz&flAssinado=N&downloadURL=/cpavPastaExterna/services/pasta-digital/arquivo'
# adjust_url3(url)

# Função 3
#go_doc_id = '7995127'
#id_liberacao_doc = 'MjczMzM5'
#get_url_download(go_doc_id, id_liberacao_doc)


<br>

## Merge PDFs


In [ ]:
def merge_files(processo_text):
    # ddd
    processo_path = data_path / processo_text
    filenames = os.listdir(processo_path)
    list_files = [
        filename for filename in filenames if filename.endswith('.pdf')]

    # Lista documentos da Pasta
    list_dict = []
    for file in list_files:
        n = file.split(' - ', 1)[0]
        list_dict.append({'id': n, 'nomearquivo': file})

    # Dataframe: files
    df = pd.DataFrame(list_dict)
    df['id'] = pd.to_numeric(df['id'])
    df.sort_values(by=['id'], inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'], inplace=True, axis=1)

    # Call the PdfFileMerger
    mergedObject = PdfFileMerger()

    for k, v in df.iterrows():
        bookmark = v['nomearquivo']
        mergedObject.append(PdfFileReader(
            processo_path / f'{v["nomearquivo"]}', 'rb'), bookmark)

    # Write all the files into a file which is named as shown below
    mergedObject.write(processo_path / f'{processo_text}.pdf')


<br>

# Rull All


In [ ]:
# Create Directory
processo_path = data_path / processo_text
processo_path.mkdir(exist_ok=True)

# Lê Tabela
df = pd.read_csv(data_path / 'tab_arquivos.csv')
df.head()


In [ ]:
# https://stackoverflow.com/questions/29563335/how-do-i-load-session-and-cookies-from-selenium-browser-to-requests-library-in-p
# Get cookies
cookies = driver.get_cookies()

# Create session from cookies
s = requests.Session()
for cookie in cookies:
    s.cookies.set(cookie['name'], cookie['value'])


In [ ]:
# Loop thought table
for k, v in df.iterrows():
    # ddddddd
    i = k+1
    url = v['url_download']
    url1 = v['url1']
    nome = v['nome']
    r = s.get(url, stream=True)

    # ddddddd
    try:
        text = r.content.decode('utf-8')
    except:
        text = 'É PDF!'

    # dddd
    if text != 'Não foi possível recuperar a liberação externa para este documento':
        print(f'Download do arquivo {i} - {nome}')
        print(url)
        print(' ')

        # Salva Arquivo
        time.sleep(5)
        filename = processo_path / f'{i} - {nome}.pdf'
        if r.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in r:
                    f.write(chunk)
                    # f.write(r.content)

    # Se for arquivo zipado entra aqui
    elif text == 'Não foi possível recuperar a liberação externa para este documento':
        print(f'Download do arquivo {i} - {nome}')
        print(url1)
        print(' ')

        # Retorna para a janela principal (fora do iframe)
        driver.switch_to.default_content()
        driver.implicitly_wait(20)

        # sss
        driver.get(url1)

        # Muda o foco para o iframe
        iframe = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]')))
        driver.switch_to.frame(iframe)

        # Clica em Download
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="btnDownload"]'))).click()

    else:
        print(f'> Erro no download do arquivo {nome}')
        print(url)
        print(' ')


In [ ]:
# print(url)
# #driver.get(url)
# r = s.get(url, stream=True)
# r


In [ ]:
# filename = processo_path / f'{i} - {nome}.pdf'
# if r.status_code == 200:
#     with open(filename, 'wb') as f:
#         for chunk in r:
#             f.write(chunk)
#             #f.write(r.content)


<br>

# ddd


In [ ]:
# Fecha Driver
driver.quit()

# Merge PDF
# merge_files(processo_text)
